In [3]:
import numpy as np
import pandas as pd
import requests
import math
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
# from sklearn.impute import SimpleImputer
# from tensorflow.keras.preprocessing.text import Tokenizer
# from sklearn.model_selection import GridSearchCV,cross_validate
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
# from sklearn.naive_bayes import MultinomialNB
import nltk
from google.cloud import storage

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pyfenix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pyfenix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pyfenix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

# Apply to all texts
# data['clean_text'] = data.text.apply(clean)
# data['clean_text'] = data['clean_text'].astype('str')

## Testing bigger df


In [20]:
big_df= pd.read_csv("../raw_data/bigger_df/steam_data.csv")
big_df.head()

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,https://store.steampowered.com/app/945360/Amon...,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...","Very Positive(4,843,904)- 87% of the 4,843,904...","Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...","Very Positive(223,706)- 80% of the 223,706 use...","Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...","Very Positive(18,951)- 92% of the 18,951 user ...","Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...","Very Positive(284,689)- 86% of the 284,689 use...","Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...


In [21]:
big_df['categories'][3]

'Single-playerOnline PvPSteam AchievementsSteam Trading CardsSteam WorkshopSteam Cloud'

In [22]:
big_df.isnull().sum()

url                0
name               5
categories       595
img_url            0
user_reviews       0
all_reviews        2
date               2
developer          2
publisher          0
price              0
pegi            4283
pegi_url           0
dtype: int64

In [23]:
big_df.dtypes

url             object
name            object
categories      object
img_url         object
user_reviews    object
all_reviews     object
date            object
developer       object
publisher       object
price           object
pegi            object
pegi_url        object
dtype: object

In [24]:
content_df = pd.read_csv('../raw_data/bigger_df/text_content.csv')
content_df.head()

,url,desc,full_desc,requirements,popu_tags
0,https://store.steampowered.com/app/945360/Amon...,An online and local party game of teamwork and...,About This GamePlay with 4-10 player online or...,System RequirementsMinimum:OS: Windows 7 SP1+P...,MultiplayerOnline Co-OpSpaceSocial DeductionSu...
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive (CS: GO) expa...,About This GameCounter-Strike: Global Offensiv...,System RequirementsWindowsMac OS XSteamOS + Li...,FPSShooterMultiplayerCompetitiveActionTeam-Bas...
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys is a massively multiplayer party gam...,About This GameFall Guys: Ultimate Knockout fl...,System RequirementsMinimum:Requires a 64-bit p...,MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
3,https://store.steampowered.com/app/1158310/Cru...,Paradox Development Studio brings you the sequ...,About This GameParadox Development Studio brin...,System RequirementsWindowsMac OS XSteamOS + Li...,StrategyRPGSimulationMedievalGrand StrategyHis...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2 is an action MMO with a single evolv...,About This GameDive into the world of Destiny ...,System RequirementsMinimum:Requires a 64-bit p...,Free to PlayLooter ShooterMultiplayerFPSAction...


In [25]:
content_df['popu_tags']

0        MultiplayerOnline Co-OpSpaceSocial DeductionSu...
1        FPSShooterMultiplayerCompetitiveActionTeam-Bas...
2        MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
3        StrategyRPGSimulationMedievalGrand StrategyHis...
4        Free to PlayLooter ShooterMultiplayerFPSAction...
                               ...                        
81043                                                    -
81044                                                    -
81045                                                    -
81046                                                    -
81047                                                    -
Name: popu_tags, Length: 81048, dtype: object

In [26]:
json_df= pd.read_json('../raw_data/bigger_df/final_data_new.json')
json_df.head()

,img_url,date,developer,publisher,full_desc,requirements,popu_tags,price,url_info,name,categories,pegi
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,"{'sort': 'game', 'desc': 'About This Game Play...",{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,{'url': 'https://store.steampowered.com/app/94...,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",NaN
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,"{'sort': 'game', 'desc': 'About This Game Coun...",{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,{'url': 'https://store.steampowered.com/app/73...,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",NaN
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,"{'sort': 'game', 'desc': 'About This Game Fall...",{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,{'url': 'https://store.steampowered.com/app/10...,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",NaN
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,"{'sort': 'game', 'desc': 'About This Game Para...",{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,{'url': 'https://store.steampowered.com/app/11...,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,"{'sort': 'game', 'desc': 'About This Game Dive...",{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,{'url': 'https://store.steampowered.com/app/10...,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...


In [27]:
json_df['full_desc'][0]['sort']

'game'

In [28]:
json_df = json_df.drop('full_desc', 1).assign(**json_df.full_desc.dropna().apply(pd.Series))

/tmp/ipykernel_452/1463184454.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  json_df = json_df.drop('full_desc', 1).assign(**json_df.full_desc.dropna().apply(pd.Series))


In [29]:
json_df['url_info'][0]

{'url': 'https://store.steampowered.com/app/945360/Among_Us/?snr=1_7_7_230_150_1',
 'id': '945360',
 'type': 'app',
 'url_name': 'Among Us'}

In [30]:
json_df = json_df.drop('url_info', 1).assign(**json_df.url_info.dropna().apply(pd.Series))

/tmp/ipykernel_452/2479464466.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  json_df = json_df.drop('url_info', 1).assign(**json_df.url_info.dropna().apply(pd.Series))


In [31]:
json_df.head()

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",NaN,game,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,945360,app,Among Us
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",NaN,game,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,730,app,CounterStrike Global Offensive
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",NaN,game,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,1097150,app,Fall Guys Ultimate Knockout
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,1158310,app,Crusader Kings III
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,1085660,app,Destiny 2


In [33]:
json_df['type'].isnull().value_counts()

False    68718
True      6103
Name: type, dtype: int64

In [34]:
json_df[json_df['type'].isnull()]

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name
56,https://steamcdn-a.akamaihd.net/steam/apps/252...,"Jul 7, 2015",Psyonix LLC,Psyonix LLC,{'minimum': {'windows': {'processor': ' 2.5 GH...,"[Multiplayer, Soccer, Competitive, Sports, Rac...",NaN,Rocket League®,"[Single-playerOnline PvPShared/Split, Screen P...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Rocket League is a high-powere...,NaN,NaN,NaN,NaN
106,https://steamcdn-a.akamaihd.net/steam/apps/252...,"Jul 7, 2015",Psyonix LLC,Psyonix LLC,{'minimum': {'windows': {'processor': ' 2.5 GH...,"[Multiplayer, Soccer, Competitive, Sports, Rac...",NaN,Rocket League®,"[Single-playerOnline PvPShared/Split, Screen P...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Rocket League is a high-powere...,NaN,NaN,NaN,NaN
538,https://steamcdn-a.akamaihd.net/steam/apps/261...,"Mar 11, 2015",Moon Studios GmbH,Xbox Game Studios,{'minimum': {'windows': {'processor': ' Intel ...,"[Great, Soundtrack, Atmospheric, Platformer, A...",NaN,Ori and the Blind Forest,"[Single-playerSteam Achievements Full, control...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game The forest of Nibel is dying. ...,NaN,NaN,NaN,NaN
619,https://steamcdn-a.akamaihd.net/steam/apps/134...,"Jul 21, 2020","PUBG Corporation, PUBG Corporation",NaN,{'minimum': {'windows': {'processor': ' Intel ...,"[Action, Adventure, Massively, Multiplayer+]",NaN,Survivor Pass: Payback,"[Online PvPDownloadable, Content StatsRemote P...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,content,About This Content Welcome back to the Jungle!...,NaN,NaN,NaN,NaN
639,https://steamcdn-a.akamaihd.net/steam/apps/901...,NaN,"Rockstar North, Rockstar Toronto",NaN,{},"[Open, World, Action, Bowling, Automobile, Mul...",NaN,Grand Theft Auto IV: Complete Edition,"[Single-playerPartial Controller, Support Inco...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game PLEASE NOTE: Microsoft no lon...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74812,https://steamcdn-a.akamaihd.net/steam/apps/134...,"29 Oct, 2020",Never Knows Best,NaN,{},"[Dating, Casual, Visual, Novel, Simulation, Wo...",NaN,恋爱绮谭~不存在的夏天~,"[Single-playerSteam is, learning about, this g...",NaN,game,About This Game FEATURE20WCGL2DSTAFFNEVER KNOW...,NaN,NaN,NaN,NaN
74814,https://steamcdn-a.akamaihd.net/steam/apps/134...,"30 Jun, 2020",zomgbab,NaN,"{'minimum': {'windows': {'processor': ' TBD', ...","[Simulation, Casual, Dark, Humor, Survival, Ho...",NaN,Social Distancing Simulator,"[Single-playerSteam Achievements StatsSteam, i...",NaN,game,About This Game Social distancing is in full e...,NaN,NaN,NaN,NaN
74817,https://steamcdn-a.akamaihd.net/steam/apps/134...,Late 2020,Lychee Game Labs,NaN,{'minimum': {'windows': {'processor': ' Core i...,"[Early, Access, Action, Roguelike, Roguevania,...",NaN,Patch Quest,"[Single-playerShared/Split Screen, Co-opFull c...",NaN,game,About This Game Patch Quest is the only action...,NaN,NaN,NaN,NaN
74818,https://steamcdn-a.akamaihd.net/steam/apps/134...,"6 Nov, 2020",七彩绘色游戏,NaN,{'minimum': {'linux': {'processor': ' Intel酷睿i...,"[Casual, Visual, Novel, Shooter, Nudity, Relax...",NaN,球球少女,"[Single-playerSteam is, learning about, this g...",NaN,game,About This Game GALGAME99CG,NaN,NaN,NaN,NaN


In [35]:
clean_df = json_df[(json_df['sort']=='game') & (json_df['url'].isnull()!= True)].copy()
clean_df.head()

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",NaN,game,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,945360,app,Among Us
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",NaN,game,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,730,app,CounterStrike Global Offensive
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",NaN,game,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,1097150,app,Fall Guys Ultimate Knockout
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,1158310,app,Crusader Kings III
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,1085660,app,Destiny 2


In [36]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42720 entries, 0 to 74820
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   img_url       42720 non-null  object
 1   date          42667 non-null  object
 2   developer     42640 non-null  object
 3   publisher     4511 non-null   object
 4   requirements  42679 non-null  object
 5   popu_tags     42698 non-null  object
 6   price         42720 non-null  object
 7   name          42720 non-null  object
 8   categories    42665 non-null  object
 9   pegi          3396 non-null   object
 10  sort          42720 non-null  object
 11  desc          42720 non-null  object
 12  url           42720 non-null  object
 13  id            42720 non-null  object
 14  type          42720 non-null  object
 15  url_name      42720 non-null  object
dtypes: object(16)
memory usage: 5.5+ MB


In [37]:
clean_df[['categories', 'popu_tags']]=clean_df[['categories', 'popu_tags']].fillna("game")

In [38]:
clean_df[clean_df['categories'].isnull()]

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name


In [39]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42720 entries, 0 to 74820
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   img_url       42720 non-null  object
 1   date          42667 non-null  object
 2   developer     42640 non-null  object
 3   publisher     4511 non-null   object
 4   requirements  42679 non-null  object
 5   popu_tags     42720 non-null  object
 6   price         42720 non-null  object
 7   name          42720 non-null  object
 8   categories    42720 non-null  object
 9   pegi          3396 non-null   object
 10  sort          42720 non-null  object
 11  desc          42720 non-null  object
 12  url           42720 non-null  object
 13  id            42720 non-null  object
 14  type          42720 non-null  object
 15  url_name      42720 non-null  object
dtypes: object(16)
memory usage: 5.5+ MB


In [40]:
clean_df[['developer', 'requirements', 'pegi','publisher']]=clean_df[['developer', 'requirements', 'pegi','publisher']].fillna("no info")

In [41]:
clean_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42720 entries, 0 to 74820
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   img_url       42720 non-null  object
 1   date          42667 non-null  object
 2   developer     42720 non-null  object
 3   publisher     42720 non-null  object
 4   requirements  42720 non-null  object
 5   popu_tags     42720 non-null  object
 6   price         42720 non-null  object
 7   name          42720 non-null  object
 8   categories    42720 non-null  object
 9   pegi          42720 non-null  object
 10  sort          42720 non-null  object
 11  desc          42720 non-null  object
 12  url           42720 non-null  object
 13  id            42720 non-null  object
 14  type          42720 non-null  object
 15  url_name      42720 non-null  object
dtypes: object(16)
memory usage: 5.5+ MB


In [42]:
clean_df.columns

Index(['img_url', 'date', 'developer', 'publisher', 'requirements',
       'popu_tags', 'price', 'name', 'categories', 'pegi', 'sort', 'desc',
       'url', 'id', 'type', 'url_name'],
      dtype='object')

In [43]:
clean_df=clean_df.drop(columns=['date','sort','id','type','url_name','price','date','requirements'])
clean_df.head()

,img_url,developer,publisher,popu_tags,name,categories,pegi,desc,url
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,"[Multiplayer, Online, Space, Social, Deduction...",Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,"[Shooter, Multiplayer, Competitive, Action, Te...",Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,"[Multiplayer, Funny, Battle, Royale, Online, F...",Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,"[Strategy, Simulation, Medieval, Grand, Strate...",Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,"[Free, Play, Looter, Shooter, Multiplayer, Act...",Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...


In [44]:
" ".join(clean_df['popu_tags'][0])

'Multiplayer Online Space Social Deduction Survival Funny2 Psychological Local Multiplayer Cartoony Casual Indie Colorful Minigames Local Down Aliens+'

In [45]:

clean_df['cat']=clean_df['categories'].apply(lambda x: " ".join(x))
clean_df.head()

,img_url,developer,publisher,popu_tags,name,categories,pegi,desc,url,cat
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,"[Multiplayer, Online, Space, Social, Deduction...",Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,"[Shooter, Multiplayer, Competitive, Action, Te...",Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,"[Multiplayer, Funny, Battle, Royale, Online, F...",Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,"[Strategy, Simulation, Medieval, Grand, Strate...",Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,"[Free, Play, Looter, Shooter, Multiplayer, Act...",Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...


In [46]:
clean_df['tags']=clean_df['popu_tags'].apply(lambda x: " ".join(x))
clean_df.head()

,img_url,developer,publisher,popu_tags,name,categories,pegi,desc,url,cat,tags
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,"[Multiplayer, Online, Space, Social, Deduction...",Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,Multiplayer Online Space Social Deduction Surv...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,"[Shooter, Multiplayer, Competitive, Action, Te...",Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...,Shooter Multiplayer Competitive Action Team- B...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,"[Multiplayer, Funny, Battle, Royale, Online, F...",Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...,Multiplayer Funny Battle Royale Online Family ...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,"[Strategy, Simulation, Medieval, Grand, Strate...",Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...,Strategy Simulation Medieval Grand Strategy Hi...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,"[Free, Play, Looter, Shooter, Multiplayer, Act...",Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...,Free Play Looter Shooter Multiplayer Action On...


In [47]:
clean_df['metadata'] = clean_df[['tags', 'cat']].apply(lambda x: ' '.join(x), axis = 1)


clean_df['metadata'] = clean_df['metadata'].apply(
    lambda x: ' '.join(list(set(x.split())))
)

In [48]:
clean_df['metadata'][0]

'Down Phone Multiplayer PvPLAN Casual Co-opCross-Platform Co-opLAN Colorful Social Play Local Funny2 Minigames Psychological Survival Tablet Online Indie Deduction on Space PvPOnline Remote Cartoony Aliens+'

In [49]:
metadata_df=clean_df.drop(columns=['popu_tags','categories','tags']).copy()

In [50]:
def clean_text(text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    
    return " ".join(lemmatized)

In [51]:
metadata_df.shape

(42720, 9)

In [52]:
metadata_df = metadata_df[
        metadata_df['desc']\
                .str.contains('About This Content', case=False) == False]


In [53]:
metadata_df.shape

(42719, 9)

In [54]:
metadata_df = metadata_df[
        metadata_df['desc']\
                .str.contains('About This Hardware', case=False) == False]

In [55]:
metadata_df.shape

(42719, 9)

In [56]:
metadata_df['clean_description'] = metadata_df.desc.astype('str').apply(clean_text)
metadata_df.head()

,img_url,developer,publisher,name,pegi,desc,url,cat,metadata,clean_description
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,Among Us,no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,Down Phone Multiplayer PvPLAN Casual Co-opCros...,game play player online via local wifi attempt...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,Counter-Strike: Global Offensive,no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...,Steam Full Shooter enabledStats Realistic Comp...,game counter strike global offensive c go expa...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,Fall Guys: Ultimate Knockout,no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...,Co-opSteam Full Phone + Friendly Party Platfor...,game fall guy ultimate knockout fling horde co...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,Crusader Kings III,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...,Steam Sexual Nudity Character Single-playerOnl...,game paradox development studio brings sequel ...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,Destiny 2,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...,Rich Co-opSteam Full a Atmospheric Shooter to ...,game dive world destiny explore mystery solar ...


In [57]:
metadata_df=metadata_df.drop(columns='desc')
metadata_df.head()

,img_url,developer,publisher,name,pegi,url,cat,metadata,clean_description
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,Among Us,no info,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,Down Phone Multiplayer PvPLAN Casual Co-opCros...,game play player online via local wifi attempt...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,Counter-Strike: Global Offensive,no info,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...,Steam Full Shooter enabledStats Realistic Comp...,game counter strike global offensive c go expa...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,Fall Guys: Ultimate Knockout,no info,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...,Co-opSteam Full Phone + Friendly Party Platfor...,game fall guy ultimate knockout fling horde co...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,Crusader Kings III,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...,Steam Sexual Nudity Character Single-playerOnl...,game paradox development studio brings sequel ...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,Destiny 2,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...,Rich Co-opSteam Full a Atmospheric Shooter to ...,game dive world destiny explore mystery solar ...


In [58]:
metadata_df=metadata_df.drop(columns='cat')
metadata_df.shape

(42719, 8)

In [59]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42719 entries, 0 to 74820
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   img_url            42719 non-null  object
 1   developer          42719 non-null  object
 2   publisher          42719 non-null  object
 3   name               42719 non-null  object
 4   pegi               42719 non-null  object
 5   url                42719 non-null  object
 6   metadata           42719 non-null  object
 7   clean_description  42719 non-null  object
dtypes: object(8)
memory usage: 2.9+ MB


In [60]:
metadata_df.describe()


,img_url,developer,publisher,name,pegi,url,metadata,clean_description
count,42719,42719,42719,42719,42719,42719,42719,42719
unique,36828,22907,1948,36314,1739,42606,32600,36057
top,https://steamcdn-a.akamaihd.net/steam/apps/105...,Choice of Games,no info,Mouse Dream,no info,https://store.steampowered.com/app/945360/Amon...,Adventure Hidden Casual Object+,game
freq,7,240,38208,7,39324,3,272,260


In [61]:
metadata_df = metadata_df.drop('pegi', 1).assign(**json_df.pegi.dropna().apply(pd.Series))

/tmp/ipykernel_452/4129431649.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  metadata_df = metadata_df.drop('pegi', 1).assign(**json_df.pegi.dropna().apply(pd.Series))


In [62]:
metadata_df.head()

,img_url,developer,publisher,name,url,metadata,clean_description,pegi_url,pegi_tags,age_limit
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,Among Us,https://store.steampowered.com/app/945360/Amon...,Down Phone Multiplayer PvPLAN Casual Co-opCros...,game play player online via local wifi attempt...,NaN,NaN,NaN
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,Steam Full Shooter enabledStats Realistic Comp...,game counter strike global offensive c go expa...,NaN,NaN,NaN
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,Fall Guys: Ultimate Knockout,https://store.steampowered.com/app/1097150/Fal...,Co-opSteam Full Phone + Friendly Party Platfor...,game fall guy ultimate knockout fling horde co...,NaN,NaN,NaN
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,Crusader Kings III,https://store.steampowered.com/app/1158310/Cru...,Steam Sexual Nudity Character Single-playerOnl...,game paradox development studio brings sequel ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, LanguagePartial, Nudity, Sexual, Theme...",m
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,Destiny 2,https://store.steampowered.com/app/1085660/Des...,Rich Co-opSteam Full a Atmospheric Shooter to ...,game dive world destiny explore mystery solar ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, Language, Violence]",m


In [64]:
metadata_df.to_csv("../raw_data/clean_bigger_df.csv")

In [65]:
metadata_df.shape

(42719, 10)

In [67]:
metadata_df['age_limit'].value_counts()

m    3395
Name: age_limit, dtype: int64